In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [17]:
!pip install sparknlp

In [18]:
!pip install pyspark

In [19]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPU
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [ ]:
trainDataset = spark.read \
  .option("header", True) \
  .csv("/kaggle/input/nlp-getting-started/train.csv")

In [ ]:
trainDataset = trainDataset.filter((trainDataset.target == 0) | (trainDataset.target == 1))
trainDataset

In [ ]:
from pyspark.sql.types import IntegerType

trainDataset = trainDataset.withColumn("target", trainDataset["target"].cast(IntegerType()))
trainDataset

In [ ]:
None_ = trainDataset.filter(trainDataset.text == None)
None_.show()
# trainDataset.where(col("text").isNull())

In [ ]:
print(np.__version__)

### Only One Hot Encoding

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.linalg import DenseVector, VectorUDT
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf, explode
from pyspark.ml.feature import SQLTransformer

# Define the pipeline stages
stages_one_hot = []

# Define the categorical columns
categorical_cols = ["keyword", "location"]

for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
    indexer.setHandleInvalid("keep")
    encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec")
    stages_one_hot += [indexer, encoder]

stages_together = []
assembler = VectorAssembler(inputCols=[col+"_vec" for col in categorical_cols],
                            outputCol="features")
stages_together += [assembler]

# Add the LogisticRegression stage with the target column as the label
# lr = LogisticRegression(featuresCol="features", labelCol="target")
# stages_together += [lr]

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["features"])\
  .setOutputCol("class")\
  .setLabelColumn("target")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)
stages_together += [classsifierdl]

# Create the pipeline
stages=stages_one_hot + stages_together
for stage in stages:
    print(f"{str(stage)} - {str(type(stage))}")
    if not (isinstance(stage, Estimator) or isinstance(stage, Transformer)):
        print(("Cannot recognize a pipeline stage of type %s." % type(stage)))
pipeline = Pipeline(stages=stages)

trainDataset = trainDataset.withColumn("target", trainDataset["target"].cast(IntegerType()))
df = trainDataset

# Fit the pipeline to the data
pipelineModel = pipeline.fit(df)

# # Apply the pipeline to the data and get the predictions
transformed_df = pipelineModel.transform(df)
transformed_df

### Everything together logistic regression

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.linalg import DenseVector, VectorUDT
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf, explode
from pyspark.ml.feature import SQLTransformer

# Define the pipeline stages
stages_one_hot = []

# Define the categorical columns
categorical_cols = ["keyword", "location"]

for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
    indexer.setHandleInvalid("keep")
    encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec")
    stages_one_hot += [indexer, encoder]

stages_text = []
# Add the DocumentAssembler stage
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
stages_text += [documentAssembler]

# Add the UniversalSentenceEncoder stage
encoder = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("sentence_embedding")
stages_text += [encoder]

stages_together = []
convert_to_vector_udf = spark.udf.register(
    "convert_to_vector_udf",
    lambda r : Vectors.dense(r[0][5]), 
    VectorUDT()
)

stages_text += [SQLTransformer(
#     statement = "SELECT convert_to_vector_udf(keyword_vec) keyword_vec_densed, convert_to_vector_udf(location_vec) location_vec_densed, convert_to_vector_udf(sentence_embedding) sentence_embedding_densed, target FROM __THIS__")]
    statement = "SELECT *, convert_to_vector_udf(sentence_embedding) sentence_embedding_densed FROM __THIS__")]

inputCols=[col+"_vec" for col in categorical_cols] + ["sentence_embedding_densed"]
# inputCols=["sentence_embedding_densed"]
assembler = VectorAssembler(inputCols=inputCols,
                            outputCol="features")
stages_together += [assembler]

# Add the LogisticRegression stage with the target column as the label
lr = LogisticRegression(featuresCol="features", labelCol="target")
stages_together += [lr]

# classsifierdl = ClassifierDLApproach()\
#   .setInputCols(["features"])\
#   .setOutputCol("class")\
#   .setLabelColumn("target")\
#   .setMaxEpochs(5)\
#   .setEnableOutputLogs(True)
# stages_together += [classsifierdl]

# Create the pipeline
stages=stages_one_hot + stages_text + stages_together
# stages=stages_text + stages_together
for stage in stages:
    print(f"{str(stage)} - {str(type(stage))}")
    if not (isinstance(stage, Estimator) or isinstance(stage, Transformer)):
        print(("Cannot recognize a pipeline stage of type %s." % type(stage)))
pipeline = Pipeline(stages=stages)

trainDataset = trainDataset.withColumn("target", trainDataset["target"].cast(IntegerType()))
df = trainDataset

# Fit the pipeline to the data
pipelineModel = pipeline.fit(df)

# # Apply the pipeline to the data and get the predictions
transformed_df = pipelineModel.transform(df)
transformed_df

In [ ]:
testDataset = spark.read \
    .option("header", True) \
    .option('multiLine', 'true') \
    .csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
predictions = pipelineModel.transform(testDataset)

In [ ]:
predictions

In [ ]:
# from pyspark.ml.linalg import Vectors, VectorUDT
# from pyspark.sql import functions as F

# ud_f = F.udf(lambda r : Vectors.dense(r), VectorUDT())
# df = df.withColumn('result_of_vector_dense', ud_f('sentence_embedding'))
# df

In [ ]:
# import pyspark.sql.functions as f
# # my_list = predictions.select(f.collect_list('sentence_embedding_dense')).first()[0]
# my_list = predictions.select("sentence_embedding_densed").rdd.flatMap(lambda x: x).collect()

In [ ]:
df_submit = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
df_predictions = predictions.toPandas()

In [ ]:
df_submit["target"] = df_predictions.prediction

In [ ]:
df_submit

In [ ]:
df_submit.to_csv("submit_all_in_pyspark_logistic_regression.csv")

In [ ]:
### Everything together - DL Classifier

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.linalg import DenseVector, VectorUDT
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf, explode
from pyspark.ml.feature import SQLTransformer

# Define the pipeline stages
stages_one_hot = []

# Define the categorical columns
categorical_cols = ["keyword", "location"]

for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
    indexer.setHandleInvalid("keep")
    encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec")
    stages_one_hot += [indexer, encoder]

stages_text = []
# Add the DocumentAssembler stage
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
stages_text += [documentAssembler]

# Add the UniversalSentenceEncoder stage
encoder = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("sentence_embedding")
stages_text += [encoder]

stages_together = []
convert_to_vector_udf = spark.udf.register(
    "convert_to_vector_udf",
    lambda r : Vectors.dense(r[0][5]), 
    VectorUDT()
)

stages_text += [SQLTransformer(
    statement = "SELECT *, convert_to_vector_udf(sentence_embedding) sentence_embedding_densed FROM __THIS__")]

inputCols=[col+"_vec" for col in categorical_cols] + ["sentence_embedding_densed"]
assembler = VectorAssembler(inputCols=inputCols,
                            outputCol="features")
stages_together += [assembler]

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["features"])\
  .setOutputCol("class")\
  .setLabelColumn("target")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)
stages_together += [classsifierdl]

# Create the pipeline
stages=stages_one_hot + stages_text + stages_together
# stages=stages_text + stages_together
for stage in stages:
    print(f"{str(stage)} - {str(type(stage))}")
    if not (isinstance(stage, Estimator) or isinstance(stage, Transformer)):
        print(("Cannot recognize a pipeline stage of type %s." % type(stage)))
pipeline = Pipeline(stages=stages)

trainDataset = trainDataset.withColumn("target", trainDataset["target"].cast(IntegerType()))
df = trainDataset

# Fit the pipeline to the data
pipelineModel = pipeline.fit(df)

# # Apply the pipeline to the data and get the predictions
transformed_df = pipelineModel.transform(df)
transformed_df

### Keyword + Text

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.linalg import DenseVector, VectorUDT
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf, explode
from pyspark.ml.feature import SQLTransformer

# Define the pipeline stages
stages_one_hot = []

# Define the categorical columns
categorical_cols = ["location"]

for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
    indexer.setHandleInvalid("keep")
    encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec")
    stages_one_hot += [indexer, encoder]

stages_text = []
# Add the DocumentAssembler stage
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
stages_text += [documentAssembler]

# Add the UniversalSentenceEncoder stage
encoder = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("sentence_embedding")
stages_text += [encoder]

stages_together = []
convert_to_vector_udf = spark.udf.register(
    "convert_to_vector_udf",
    lambda r : Vectors.dense(r[0][5]), 
    VectorUDT()
)

stages_text += [SQLTransformer(
#     statement = "SELECT convert_to_vector_udf(keyword_vec) keyword_vec_densed, convert_to_vector_udf(location_vec) location_vec_densed, convert_to_vector_udf(sentence_embedding) sentence_embedding_densed, target FROM __THIS__")]
    statement = "SELECT *, convert_to_vector_udf(sentence_embedding) sentence_embedding_densed FROM __THIS__")]

inputCols=[col+"_vec" for col in categorical_cols] + ["sentence_embedding_densed"]
# inputCols=["sentence_embedding_densed"]
assembler = VectorAssembler(inputCols=inputCols,
                            outputCol="features")
stages_together += [assembler]

# Add the LogisticRegression stage with the target column as the label
lr = LogisticRegression(featuresCol="features", labelCol="target")
stages_together += [lr]

# classsifierdl = ClassifierDLApproach()\
#   .setInputCols(["features"])\
#   .setOutputCol("class")\
#   .setLabelColumn("target")\
#   .setMaxEpochs(5)\
#   .setEnableOutputLogs(True)
# stages_together += [classsifierdl]

# Create the pipeline
stages=stages_one_hot + stages_text + stages_together
# stages=stages_text + stages_together
for stage in stages:
    print(f"{str(stage)} - {str(type(stage))}")
    if not (isinstance(stage, Estimator) or isinstance(stage, Transformer)):
        print(("Cannot recognize a pipeline stage of type %s." % type(stage)))
pipeline = Pipeline(stages=stages)

trainDataset = trainDataset.withColumn("target", trainDataset["target"].cast(IntegerType()))
df = trainDataset

# Fit the pipeline to the data
pipelineModel = pipeline.fit(df)

# # Apply the pipeline to the data and get the predictions
transformed_df = pipelineModel.transform(df)
transformed_df

In [ ]:
testDataset = spark.read \
    .option("header", True) \
    .option('multiLine', 'true') \
    .csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
predictions = pipelineModel.transform(testDataset)

In [ ]:
df_submit = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
df_predictions = predictions.toPandas()

In [ ]:
df_submit["target"] = df_predictions.prediction

In [ ]:
df_submit.to_csv("submit_keyword_and_text_pyspark_logistic_regression.csv")